In [1]:
/*
 *  This Jupyter-Kotlin notebook was created for my personal reseach of DeFi and MEV.
 *
 *  Table of contents:
 *  ------------------------------------------
 *  1. Overview of MEV and main MEV startegies
 *  2. Arbitrage
 *  3. Liquidations
 *  4. Sandwich trading
 *  5. Long tail strategies
 *  6. Uniswap V2
 *  -----------------------
 *  ... future additions
 * 
 *  Created by Filip Zupančič | EncryptIQ
 *  Sponsored by Kriptal | https://kriptal.io/
*/

In [6]:
// The notebook also includes some (more or less) useful 
// Kotlin code snippets to get familiar with the syntax.
class Greeter(val name: String) {
    fun greet() {
        println("Hello, $name! Let's dive into MEV.")
    }
}

Greeter("Filip").greet()

Hello, Filip! Let's dive into MEV.


In [ ]:
/* 
 *    MEV - A short overview | https://ethereum.org/en/developers/docs/mev/
 *
 *    Maximal extractable value (MEV) refers to the maximum value that can be extracted from block production in excess 
 *    of the standard block reward and gas fees by including, excluding, and changing the order of transactions in a block.
 *
 *    proof-of-work : miners control transaction inclusion, exclusion, and ordering (miner extractable value)
 *    proof-of-stake : validators take the role of miners (maximal extractable value)
 *
 *    In theory MEV accrues entirely to miners/validators. 
 *    In practice a portion of MEV is extracted by independent network participants referred to as "searchers."
 *
 *   "gas golfing" — programming transactions so that they use the least amount of gas — a competitive advantage. 
 *   It allows searchers to set a higher gas price while keeping their total gas fees constant (gas fees = gas price * gas used).
 *
 *   Gas golfing techniques
 *   -----------------------
 *   1.) 
 *   using addresses that start with a long string of zeroes (e.g. 0x0000000000C521824EaFf97Eac7B73B084ef9306) 
 *   cause they take less space (and hence gas) to store;
 *
 *   2.)
 *   leaving small ERC-20 token balances in contracts, since it costs more gas to initialize a storage slot 
 *   (the case if the balance is 0) than to update a storage slot.
 *
 *
 *   Generalized frontrunners:
 *   -------------------------
 *   Rather than programming complex algorithms to detect profitable MEV opportunities, some searchers run generalized frontrunners. 
 *   Generalized frontrunners are bots that watch the mempool to detect profitable transactions.
 *
 *
 *   Flashbots: 
 *   ----------
 *   an independent project which extends the go-ethereum client with a service that allows searchers to submit MEV transactions to miners without revealing them to the public mempool. 
 *   This prevents transactions from being frontrun by generalized frontrunners. Currently a significant portion of MEV transactions is routed through Flashbots.
 */

In [ ]:
/*
 *  DEX Arbitrage
 *  --------------
 *  The general idea is to take advantage of mispricing between exchanges. When someone executes a large trade
 *  in to one liquidity pool it can create an inbalance distorting the price and causing slippage for that trader.
 *  Arbitrage bots will then work to restore the balance by taking liquidity from other markets.
 *
 *  CEX
 *  --------------
 *  A centralized exchange, such as Binance, CEX.io, Kraken, or OKEx, has its own order book. In this, every order is recorded and validated. 
 *  To ensure correctness, data is exchanged internally via dedicated servers and goes through centralized security processes. As a rule, CEXs 
 *  operate under regulatory supervision and have extensive know-your-customer policies built-in.
 * 
 *  DEX
 *  --------------
 *  DEXs facilitate peer-to-peer trading by relying on automated smart contracts to execute trades without an intermediary.
 *  DEXs handle this in one of three ways: an on-chain order book, an off-chain order book, or an automated market maker approach.
 *
 *  How arbitrage works | https://www.youtube.com/watch?v=wn8r674U1B4
 *  1.) Monitor the prices on different exchanges
 */

In [ ]:
/*
 *  Liquidations
 *  -------------
 * 
 *
 *
 *
 *
 *
 *
 *
 */

In [ ]:
/*
 *  Sandwiching
 *  ------------
 *
 *
 */

In [ ]:
/* 
Uniswap V2

Automated Market Makers (AMMs)
-------------------------------
decentralized exchanges that create pools of liquidity (tokens) deposited by users and price the assets within the pool using algorithms.

Uniswap V1 and V2 work via a constant-product formula:

x * y = k

where
x: number of token 1 in the pool
y: number of token 2 in the pool
k: constant that stays constant during trades but is recalculated when liquidity is provided / withdrawn

liquidity (LP)
--------------

Spot Price (SP)
---------------
a constant-product AMM like Uniswap is calculated by dividing the amount of token 1 in the pool by the amount of token 2.
SP = x / y

Effective Price (EP)
--------------------
of a trade is the amount of token received over the amount given away.

Impermanent Loss (IL)
---------------------
occurs when the price of the tokens changes after you’ve deposited them in a pool. When price moves along the curve, 
the Liquidity Provider’s (LP’s) position loses value.

*/

In [2]:
import kotlin.math.*

class Uniswapper() {
    /*
     * Function swapTokens simulates the swap of two tokens
     * amountA: Double represents the amount of input token
     * tokenA: String represents the name of the input token
     * tokenB: String represents the name of the output token
     */
    //private var poolReserveA : Double = 1200
    //private var poolReserveB : Double = 400
    val poolReserves = mutableMapOf("tokenA" to 1200.0, "tokenB" to 400.0) 
    
    fun swapTokens(amountA: Double, tokenA: String, tokenB: String): Double {
        println("poolReserves: $poolReserves")
        val oldK = poolReserves["tokenA"]!!*poolReserves["tokenB"]!!
        println("Old k: $oldK")
        val poolReservesA = poolReserves["tokenA"]
        val poolReservesB = poolReserves[tokenB]
        val priceB = poolReservesA!! / poolReservesB!!
        val amountB = amountA / priceB
        poolReserves["tokenA"] = poolReservesA!! + (amountA + (amountA*0.03))
        poolReserves["tokenB"] = poolReservesB!! - (amountB)
        println("Swapping $amountA of $tokenA for $amountB of $tokenB.")
        println("poolReserves: $poolReserves")
        val newK = poolReserves["tokenA"]!!*poolReserves["tokenB"]!!
        println("New k: $newK")
        return poolReserves["tokenA"]!!*poolReserves["tokenB"]!!
    }
}

Uniswapper().swapTokens(3.0, "tokenA", "tokenB")

poolReserves: {tokenA=1200.0, tokenB=400.0}
Old k: 480000.0
Swapping 3.0 of tokenA for 1.0 of tokenB.
poolReserves: {tokenA=1203.09, tokenB=399.0}
New k: 480032.91


480032.91

In [7]:
%use lets-plot

n = 2000

val data = mapOf (
    "Amount token A" to listOf( 1200.0, 1203.03 ),
    "Amount token B" to listOf( 400.0, 399.0 )
)


var p = lets_plot(data) { x = "Amount token A" } +
        geomPoint { y = "Amount token B" } +
        geomLine { y = "Amount token B" } +
        ggsize(500, 250)
        
p

//ggsave(p, "density.png")

In [17]:
import kotlin.random.Random

%use lets-plot

// Let's calculate the price changes
//val tokenAarr = ArrayList<Double>()
//val tokenBarr = ArrayList<Double>()
val tokenAarr: MutableList<Double> = mutableListOf()
val tokenBarr: MutableList<Double> = mutableListOf()
var reservesA = 1200.0
var reservesB = 300.0
val k = reservesA * reservesB
var priceB = 0.0
var priceA = 0.0
var amountA = 0.0
var amountB = 0.0

// Let's simulate 2.000.000 swaps
// constant formula x * y = k
for (i in 1..1000000) {
    // First we're simulating a swap of tokenA (input) for tokenB (output)
    priceB = reservesA / reservesB
    amountA = Random.nextDouble(from = 0.0, until = reservesB*priceB)
    if(reservesB - (amountA / priceB) > 1.0) {
        reservesA = reservesA + (amountA + amountA*0.03)
        reservesB = reservesB - (amountA / priceB)
        tokenAarr.add(reservesA)
        tokenBarr.add(reservesB)
    }
    
    // Now we're simulating a swap of tokenB (input) for tokenA (output)
    priceA = reservesB / reservesA
    amountB = Random.nextDouble(from = 0.0, until = reservesA*priceA)
    if (reservesA - (amountB / priceA) > 1.0) {
        reservesB = reservesB + (amountB + amountB*0.03)
        reservesA = reservesA - (amountB / priceA)
        tokenAarr.add(reservesA)
        tokenBarr.add(reservesB) 
    }
}

//println(tokenAarr)
//println(tokenBarr)

//tokenAarr = asFArray(tokenAarr)
//tokenBarr = asFArray(tokenBarr)

val data = mapOf (
    "Amount token A" to tokenAarr,
    "Amount token B" to tokenBarr
)


var p = lets_plot(data) { x = "Amount token A" } +
        geomPoint { y = "Amount token B" } +
        ggsize(1000, 500)
        
p

In [10]:
/*
Uniswap V3



*/